In [ ]:
import os
import json
import string
import time
import wave
import sys

from dotenv import load_dotenv

import azure.cognitiveservices.speech as speechsdk

import openai
from openai.embeddings_utils import get_embedding, cosine_similarity



In [ ]:
load_dotenv()

openai.api_type = "azure"
openai.api_key = os.getenv('OPENAI_API_KEY') 
openai.api_base = os.getenv('OPENAI_API_BASE') 
openai.api_version = "2022-06-01-preview"

In [ ]:
SPEECH_KEY = os.getenv["SPEECH_API_KEY"]
COMPLETIONS_MODEL = os.getenv["COMPLETIONS_MODEL"]
SERVICE_REGION = os.getenv["SERVICE_REGION"]